In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ParameterGrid

### Load the datasets

In [73]:
def load_meta_data(data_file_name,dataset_column_names,x_column_names,to_categorical_column_names,metric_name,error_metric=True):
    dataset=pd.read_csv(data_file_name,names=dataset_column_names)
    y=np.zeros(len(dataset))
    print(len(y))
    #CALCULATE THE Y
    if error_metric:
        min_error=dataset.loc[dataset[metric_name].idxmin()][metric_name]
        error_ratio=min_error/dataset[metric_name]
        y=error_ratio
    else:
        max_acc=dataset.loc[dataset[metric_name].idxmax()][metric_name]
        acc_ratio=max_acc/dataset[metric_name]
        y=acc_ratio
    
    x=dataset[x_column_names]

    dummies = pd.get_dummies(x[to_categorical_column_names[:]])
    x=x.drop(to_categorical_column_names,axis=1)
    x=pd.concat([x,dummies],axis=1)
    x.head()
    return x,y

dataset_column_names=["architecture","task","num_features","training_samples",
            "n_layers", "input_shape","activation_function",
            "learning_rate", "batch_size", "loss","fit_time","mae"]
x_column_names=["num_features","training_samples",
                        "n_layers","activation_function",
                        "learning_rate", "batch_size",]
metric_name="mae"
to_categorical_column_names=["activation_function"]
data_file_name="./data/1d_irnet.csv"


x,y=load_meta_data(data_file_name,dataset_column_names,x_column_names,to_categorical_column_names,metric_name,error_metric=True)                              




30


### Meta learner

In [74]:
regr = RandomForestRegressor(random_state=0)
regr.fit(x,y)

#HYPERPARAMETERS TO EVALUATE
num_features=[29]
training_samples=[240122]
n_layers=[1,2,3]
learning_rate=[0.01,0.001,0.0001,0.00001]
batch_size=[16,32,64,128]
activation_function=['relu','elu','tanh','sigmoid']

#CREATE THE HYPERPARAMETER SPACE
dict_all_hyperparams=dict(num_features=num_features,
                            training_samples=training_samples,
                            n_layers=n_layers,
                            learning_rate=learning_rate,
                            batch_size=batch_size,
                            activation_function=activation_function)

grid_search_population=pd.DataFrame(ParameterGrid(dict_all_hyperparams))

#PREPROCESS THE DATA TO BE PREDICTED BY THE METALEARNER
dummies2 = pd.get_dummies(grid_search_population[to_categorical_column_names[:]])
x_test=pd.concat([grid_search_population[x_column_names],dummies2],axis=1)
x_test=x_test.drop(to_categorical_column_names,axis=1)



In [75]:
#PREDICTION OF THE HYPERPARAMETER SPACE
predictions= pd.DataFrame(regr.predict(x_test))
x_test["y"]=pd.DataFrame(regr.predict(x_test))
x_test.head()

x_test=x_test.sort_values("y",ascending=False)
x_test.head()

,num_features,training_samples,n_layers,learning_rate,batch_size,activation_function_elu,activation_function_relu,activation_function_sigmoid,activation_function_tanh,y
37,29,240122,2,0.01,128,0,1,0,0,0.983194
26,29,240122,3,0.01,64,0,1,0,0,0.982024
38,29,240122,3,0.01,128,0,1,0,0,0.981702
25,29,240122,2,0.01,64,0,1,0,0,0.981559
122,29,240122,3,0.01,64,0,0,0,1,0.976312
